Images need to be normalized from 0 to 1, relative to the overal intensity of the 1024x1024 image not the overall intensity of an individual patch

In [13]:
from scipy import ndimage
import numpy as np
from pathlib import Path

def random_rot_flip(image, label):
    k = np.random.randint(0, 4)
    image = np.rot90(image, k)
    label = np.rot90(label, k)
    axis = np.random.randint(0, 2)
    image = np.flip(image, axis=axis).copy()
    label = np.flip(label, axis=axis).copy()
    return image, label

def zip_and_augment(labelfolder, outfolder):
    if (outfolder.exists() == False):
        outfolder.mkdir()
    counter = 0
    names = []
    for label_name in labelfolder.glob("*.npz"):

        imglabel = np.load(label_name)
        img = imglabel["image"]
        lbl = imglabel["label"]
        img, lbl = random_rot_flip(img,lbl)

        imageaddress = outfolder / ("augmented_"+str(counter))
        names.append("augmented_"+str(counter))
        np.savez(imageaddress, image=img, label=lbl)
        counter+=1

        img, lbl = random_rot_flip(img,lbl)
        imageaddress = outfolder / ("augmented_"+str(counter))
        names.append("augmented_"+str(counter))
        np.savez(imageaddress,image=img, label=lbl)
        counter+=1
    with open("augmented_names.txt", "w") as f:
        f.write("\n".join(names))

    return

In [14]:
zip_and_augment(Path("./Data/npzs"), Path("./Data/augmented/"))